# Instalação de bibliotecas e imports iniciais

In [ ]:
!pip install PyMuPDF Pillow openai

import os
import json
import fitz
import time
import base64
from PIL import Image
from io import BytesIO
from openai import OpenAI
from google.colab import userdata


print("Configuração inicial e bibliotecas instaladas.")

Configuração inicial e bibliotecas instaladas.


# Conexão com a API

In [ ]:
NEMOTRON_BASE_URL = "https://openrouter.ai/api/v1"
NEMOTRON_API_KEY = userdata.get('NEMOTRON_API_KEY')
NEMOTRON_MODEL_NAME = "nvidia/nemotron-nano-12b-v2-vl:free"

try:
  client = OpenAI(
      base_url=NEMOTRON_BASE_URL,
      api_key=NEMOTRON_API_KEY,
  )
  print("Cliente API inicializado com sucesso!")
except Exception as e:
  print(f"Erro ao inicializar o cliente API: {e}")

Cliente API inicializado com sucesso!


# Funções auxiliares

In [ ]:
def convert_pdf_page_to_image(pdf_path, page_num, output_image_path, dpi=300):
    # converte uma página do pdf em imagem png
    try:
        doc = fitz.open(pdf_path)

        # confere existência da página
        if page_num >= doc.page_count:
            print(f"erro: página {page_num+1} não existe (pdf tem {doc.page_count})")
            return None

        # procedimento padrão de conversão de imagem
        page = doc.load_page(page_num)
        zoom = fitz.Matrix(dpi / 72, dpi / 72)
        pix = page.get_pixmap(matrix=zoom)
        pix.save(output_image_path)
        doc.close()
        return output_image_path

    # tratamento de exceções
    except Exception as e:
        print(f"erro ao converter pdf: {e}")
        return None

def image_to_base64(image_path):
    # converte imagem local em base64
    try:
        with open(image_path, "rb") as f:
            return base64.b64encode(f.read()).decode("utf-8")
    # tratamento de exceções
    except Exception as e:
        print(f"erro ao codificar imagem: {e}")
        return None

# Função de extração de dados utilizando o modelo nemotron_nano_12b_v2_vl:free

In [ ]:
def extract_data_with_nemotron_real(image_path, prompt_text, json_schema_str):
    global client, NEMOTRON_MODEL_NAME
    print(f"\n--- Inferência real: {image_path} ---")

    # confere existência da imagem
    image_base64 = image_to_base64(image_path)
    if not image_base64:
        return {"status": "falha", "motivo": "imagem não carregada"}

    # prompt reforçando as instruções para o modelo
    reinforced_prompt = f"""
    {prompt_text}

    siga exatamente o formato json abaixo, sem texto extra, markdown ou comentários.
    schema esperado:
    {json_schema_str}
    """

    messages = [
        {"role": "user", "content": [
            {"type": "text", "text": reinforced_prompt},
            {"type": "image_url", "image_url": {"url": f"data:image/jpeg;base64,{image_base64}"}},
        ]}
    ]

    try:
        start_time = time.time() # início da medição de tempo
        response = client.chat.completions.create(
            model=NEMOTRON_MODEL_NAME,
            messages=messages,
            temperature=0.0,
            max_tokens=2048
        ) # requisição para o modelo
        end_time = time.time() # fim da medição de tempo
        raw_output = response.choices[0].message.content.strip()
        print(f"latência total: {end_time - start_time:.2f}s") # cálculo da latência total

        # conferência do formato json
        try:
            return json.loads(raw_output)
        # tratamento de erros no json
        except json.JSONDecodeError:
            print("erro de parsing, tentando reparo...")
            repaired = raw_output.replace("```json", "").replace("```", "").strip()
            if not repaired.startswith("{"):
                repaired = repaired[repaired.find("{"):]
            if not repaired.endswith("}"):
                repaired = repaired[:repaired.rfind("}") + 1]
            try:
                return json.loads(repaired)
            # retorno da saída pura, impossível gerar json
            except json.JSONDecodeError:
                print("json irrecuperável. salvando saída bruta.")
                return {"status": "parsing_falhou", "raw_output": raw_output}

    # tratamento de exceções
    except Exception as e:
        return {"status": "erro_api", "motivo": str(e)}

# Execução do primeiro caso de teste (extração de dados CNH)

In [ ]:
# CASO 1: Documento CNH -> JPEG
print("Caso de uso 1")

prompt_cnh = ("""
você é um agente de inteligência documental visual especializado em documentos de identidade brasileiros (CNH).
analise a imagem e extraia os valores exatos dos campos, com base na posição visual do texto na imagem.

instruções obrigatórias:
1. leia o documento da esquerda para a direita e de cima para baixo.
2. use os rótulos impressos no documento (ex: 'Nome', 'Filiação', 'Data de Nascimento', 'Registro', etc.) para localizar o valor correspondente logo à direita ou abaixo.
3. se um campo estiver ilegível, retorne null (não invente).
4. mantenha o formato exato dos valores conforme o schema json.
5. retorne apenas o objeto json puro.

campos a localizar:
- nome_completo → linha superior central, após o rótulo “Nome”
- cpf → canto superior direito, próximo a foto
- data_nascimento → logo ao lado do cpf na seção de dados pessoais
- data_emissao → parte inferior direita, campo “Data Emissão imediatamente ao lado do local”
- data_primeira_habilitacao → campo mais a direita logo acima do campo "Observações"
- filiacao_pai → linha após “Filiação”, primeira posição
- filiacao_mae → linha após “Filiação”, segunda posição
""")

CNH_SCHEMA = {
    "nome_completo": "string",
    "cpf": "string (XXX.XXX.XXX-XX)",
    "data_nascimento": "string (DD/MM/AAAA)",
    "data_emissao": "string (DD/MM/AAAA)",
    "data_primeira_habilitacao": "string",
    "filiacao_pai": "string",
    "filiacao_mae": "string"
}

CNH_IMAGE_PATH = "Documento 1.jpeg"
start = time.time()
dados_extraidos_cnh = extract_data_with_nemotron_real(CNH_IMAGE_PATH, prompt_cnh, CNH_SCHEMA)
latencias.append(time.time() - start)
print("RESULTADO DA POC: CASO 1 (CNH)")
print(json.dumps(dados_extraidos_cnh, indent=4, ensure_ascii=False))

Caso de uso 1

--- Inferência real: Documento 1.jpeg ---
latência total: 12.82s
RESULTADO DA POC: CASO 1 (CNH)
{
    "nome_completo": "LINCE DA SILVA",
    "cpf": "001.243.607-00",
    "data_nascimento": "06/08/1981",
    "data_emissao": "02/06/2018",
    "data_primeira_habilitacao": "06/08/2017",
    "filiacao_pai": "Pai José da Silva",
    "filiacao_mae": "Mãe Maria da Silva"
}


# Execução do segundo caso de teste (extração de dados de pdf extenso)

In [ ]:
# CASO 2: Documento Extenso (PDF 42 páginas -> JSON de Tabela)
print("Caso de uso 2")

prompt_documento_complexo = ("""
Você é um agente de inteligência documental.
Sua tarefa é analisar a página fornecida e extrair o conteúdo textual e visual, preservando a ordem em que os elementos aparecem.

Regras obrigatórias:
1. A resposta DEVE ser um objeto JSON **válido**, sem comentários ou texto fora da estrutura JSON.
2. Use exatamente o SCHEMA abaixo.
3. Cada elemento da lista `conteudo_pagina` deve representar um bloco distinto da página, na ordem em que aparece visualmente.
4. Tipos possíveis: 'texto', 'tabela', 'figura'.
5. Se algum campo não existir, use `null` ou `[]`.
6. Para tabelas:
   - Liste os cabeçalhos no campo `colunas` como uma lista de strings.
   - Liste as linhas no campo `dados`, cada uma também como uma lista de strings.
7. Descreva brevemente as figuras e forneça uma interpretação dos dados mostrados.

SCHEMA JSON:
{
  "numero_pagina": "integer",
  "titulo_secao": "string",
  "conteudo_pagina": [
    {
      "tipo": "string (ex: 'texto', 'tabela', 'figura')",
      "titulo": "string (opcional)",
      "texto": "string (usado se tipo='texto')",
      "tabela": {
        "colunas": ["string"],
        "dados": [
          ["string"]
        ]
      },
      "figura": {
        "descricao_visual": "string",
        "interpretacao": "string"
      }
    }
  ]
}

Analise visualmente a página e extraia:
- parágrafos e descrições textuais;
- tabelas (mantendo colunas e linhas na mesma ordem);
- figuras (descrição e interpretação).

A ordem dos elementos no JSON deve refletir a ordem visual na página.
Responda apenas com o objeto JSON.
""")


EXTENSO_SCHEMA = {
  "numero_pagina": "integer",
  "titulo_secao": "string",
  "conteudo_pagina": [
    {
      "tipo": "string (ex: 'texto', 'tabela', 'figura')",
      "titulo": "string (opcional)",
      "texto": "string (usado se tipo='texto')",
      "tabela": {
        "colunas": ["string"],
        "dados": [
          ["string"]
        ]
      },
      "figura": {
        "descricao_visual": "string",
        "interpretacao": "string"
      }
    }
  ]
}

EXTENSO_SCHEMA_STR = json.dumps(EXTENSO_SCHEMA, indent=2)

# Caminhos
PDF_PATH = "Documento 3.pdf"
PAGE_TO_EXTRACT = 21
TEMP_IMAGE_PATH = "PDF_Page_Temp.png"

# PRÉ-PROCESSAMENTO OBRIGATÓRIO (PDF -> Imagem)
print(f"Iniciando pré-processamento do PDF...")
converted_image_path = convert_pdf_page_to_image(
    pdf_path=PDF_PATH,
    page_num=PAGE_TO_EXTRACT,
    output_image_path=TEMP_IMAGE_PATH
)

if converted_image_path:
    # CHAMADA REAL À API com a IMAGEM convertida
    prompt_extenso = "Analise a página e extraia todo o texto dela, além disso de uma descrição detalhada da tabela"
    dados_extraidos_extenso = extract_data_with_nemotron_real(converted_image_path, prompt_documento_complexo, EXTENSO_SCHEMA_STR)

    print("RESULTADO DA POC: CASO 2 (PDF -> JSON)")
    print(json.dumps(dados_extraidos_extenso, indent=4, ensure_ascii=False))

else:
    print("POC do Caso 2 não executada devido a falha no pré-processamento do PDF.")

Caso de uso 2
Iniciando pré-processamento do PDF...

--- Inferência real: PDF_Page_Temp.png ---
latência total: 13.54s
erro de parsing, tentando reparo...
RESULTADO DA POC: CASO 2 (PDF -> JSON)
{
    "numero_pagina": 1,
    "titulo_secao": "Figures 15 and 16. Claude 3 Opus substantially outperforms all other models and gets close to perfect performance on this task, with a 99.4% average recall, and maintaining a 98.3% average recall at 200k context length. The results are shown in Table 7.",
    "conteudo_pagina": [
        {
            "tipo": "texto",
            "texto": "Figures 15 and 16. Claude 3 Opus substantially outperforms all other models and gets close to perfect performance on this task, with a 99.4% average recall, and maintaining a 98.3% average recall at 200k context length. The results are shown in Table 7."
        },
        {
            "tipo": "tabela",
            "titulo": "Table 7",
            "colunas": [
                "Matrices",
                "Sonnet",

# Execução do terceiro caso de teste (extração de dados fatura)

In [ ]:
# CASO 3: Layout Complexo (Fatura de Energia - JPEG)

prompt_fatura_reduzido = ('''Você é um Document Intelligence Agent especializado em faturas de energia elétrica.

Sua tarefa é analisar a imagem fornecida e **extrair exclusivamente os valores correspondentes** aos campos do SCHEMA JSON abaixo.

REGRAS OBRIGATÓRIAS:
1. A saída DEVE ser um objeto JSON completo e válido, **começando com '{' e terminando com '}'**, sem texto fora da estrutura JSON.
2. Mantenha a **hierarquia exata** e os **nomes de chaves** do schema.
3. Se algum campo não existir na imagem, use `null`.
4. Para valores monetários:
   - Use ponto como separador decimal (ex: 63.72)
   - Não inclua "R$"
5. Para datas, use o formato **DD/MM/AAAA**.
6. Para CNPJ, CPF e números de contrato, preserve a formatação exibida no documento.
7. O campo `total_a_pagar_reais` deve ser um número FLOAT, e não string.''')

print("Caso de uso 3")


FATURA_SCHEMA = {
  "tipo_documento": "string",
  "empresa_emissora": {
    "razao_social": "string",
    "cnpj": "string"
  },
  "dados_cliente": {
    "nome_cliente": "string",
    "cpf": "string",
    "endereco": "string"
  },
  "resumo_fatura": {
    "conta_contrato": "string",
    "mes_referencia": "string",
    "data_vencimento": "string",
    "total_a_pagar_reais": "float"
  },
  "consumo": {
    "quantidade_kwh": "string",
    "preco_unitario": "string",
    "valor_total_reais": "string"
  },
  "tributos": {
    "icms_percentual": "string",
    "pis_percentual": "string",
    "cofins_percentual": "string"
  },
  "rodape": {
    "codigo_barras": "string"
  }
}

FATURA_SCHEMA_STR = json.dumps(FATURA_SCHEMA, indent=2)
FATURA_IMAGE_PATH = "Documento 2.jpg"

prompt_fatura = "Extraia todas as informações presentes na fatura de luz"
dados_extraidos_fatura = extract_data_with_nemotron_real(FATURA_IMAGE_PATH, prompt_fatura, FATURA_SCHEMA_STR)

print("RESULTADO DA POC: CASO 3 (Fatura)")
print(json.dumps(dados_extraidos_fatura, indent=4, ensure_ascii=False))

Caso de uso 3

--- Inferência real: Documento 2.jpg ---
latência total: 27.56s
RESULTADO DA POC: CASO 3 (Fatura)
{
    "tipo_documento": "Nota Fiscal",
    "empresa_emissora": {
        "razao_social": "CELEPE - Grupo Neoenergia",
        "cnpj": ""
    },
    "dados_cliente": {
        "nome_cliente": "Maria Jose da Silva",
        "cpf": "123.456.789-10",
        "endereco": "RUA JOSÉ FERNANDES VIEIRA 175 BOA VISTA"
    },
    "resumo_fatura": {
        "conta_contrato": "1234567890",
        "mes_referencia": "10/07/2014",
        "data_vencimento": "10/07/2014",
        "total_a_pagar_reais": 63.72
    },
    "consumo": {
        "quantidade_kwh": "160.0000000",
        "preco_unitario": "0,39825378",
        "valor_total_reais": "63,72"
    },
    "tributos": {
        "icms_percentual": "53,72%",
        "pis_percentual": "25,32%",
        "cofins_percentual": "4,54%"
    },
    "rodape": {
        "codigo_barras": "8396800000007110053680000118223358000002050579722655"
    }
}
